In [207]:
# !pip install python-binance

In [1]:
from binance import Client
from datetime import datetime, timedelta
client = Client(api_key="jM9zEeTTzT7R7QGlNrh5tUpOc8l71hOxu7CewCL4M0NvbBp3ilgmEwMOOjJfCRVB",api_secret="iZxFmlz5bvROsHQPZTP5hlJtsxYjOu8wocOlLWzIDdhMzEQu7KyyHB2DseOYQ66s",testnet=True)

import os
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine, text
import pg8000.native
import pandas as pd

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

connector = Connector()

def getconn() -> pg8000.native.Connection:
    conn: pg8000.native.Connection = connector.connect(
        "semiotic-pact-388619:us-central1:tradingbot",
        "pg8000",
        user="postgres",
        password="GuruSat.3",
        db="criptos"
    )
    return conn

connection = create_engine('postgresql+pg8000://',creator=getconn)

In [2]:
def MACD(df):
    df['EMA12'] = df.Precio.ewm(span=12).mean()
    df['EMA26'] = df.Precio.ewm(span=26).mean()
    df['MACD'] = df.EMA12 - df.EMA26
    df['Signal'] = df.MACD.ewm(span=9).mean()
    # print('Indicadores agregados')

def señal_MACD(df):
    buy, sell = [],[]
    for i in range(7,len(df)):
        if df.MACD.iloc[i] > df.Signal.iloc[i] and df.MACD.iloc[i-1] < df.Signal.iloc[i-1]:
            buy.append(i)
        elif df.MACD.iloc[i] < df.Signal.iloc[i] and df.MACD.iloc[i-1] > df.Signal.iloc[i-1]:
            sell.append(i)
    return buy, sell

def slopee(x1,y1,x2,y2):
    x = (y2 - y1) / (x2 - x1).total_seconds()
    return x

In [29]:
tiempos_salida = []
precios_salida = []
tiempos_entrada = []
precios_entrada = []
buy_ids = []
sell_ids = []
balances = []
ganancia_maxima = 0
balance_inicial = float(client.get_account()['balances'][6]['free'])

#### PARÁMETROS DE LAS CONDICIONES DE SALIDA Y ENTRADA.
PAIR = 'BNBUSDT'
pair = 'bnb'
LOCALIDAD_SEÑAL_ENTRADA = 60 # EN SEGUNDOS. param_1
# El primero de ellos es el promedio de la pendiente (precio/tiempo) de las últimas 10 filas del dataframe. Se utiliza para 
# saber si se debe entrar en una posición corta al alza (puesto que a la baja no estamos tradeando) junto con otra condición
PENDIENTE_ENTRADA = 0 # param_2
# LOS UMBRALES SON PORCENTAJES.
UMBRAL_PERDIDA_1P = -0.0087 # param_3
UMBRAL_GANANCIA_1G = 0.025 # param_4
UMBRAL_GANANCIA_3G = 0.05 # param_5
UMBRAL_GANANCIA_4G = 0.005 # param_6
UMBRAL_MAXIMO_2G = 0.025 # param_7
UMBRAL_MINIMO_2G = 0.005 # param_8
LOCALIDAD_SEÑAL_SALIDA_2G = 20 # EN SEGUNDOS param_9
LIMITE_TIEMPO_POSITIVO_4G = 10 # EN MINUTOS param_10

path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/trade_logs/testnet/MACD_logicv1/'

while True:
    day = datetime.utcnow() - timedelta(1)

    with connection.connect() as conn:
        df =conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
    df = pd.DataFrame(df)[['Tiempo', 'Precio']].set_index('Tiempo')
    df = df.sort_values(by='Tiempo')
    MACD(df)
    # print(datetime.utcnow() - df.index[-1])
    buy, sell = señal_MACD(df)

    buyprices = df.Precio.iloc[buy]
    sellprices = df.Precio.iloc[sell]

##############  VERIFICAMOS SI NO HAY OPERACIONES ABIERTAS: Como solo vamos a trabajar con señales de entrada, toda posición
##############  está cerrada si la siguiente condición se cumple.

    if client.get_all_orders(symbol=PAIR, requests_params={'timeout': 30})[-1]['side'] == 'SELL':

##############  EN CASO DE NO HABER, SE PROCEDE A VERIFICAR LA "LOCALIDAD" DE LA SEÑAL: En caso de ser una señal local se crea 
##############  una orden de compra y se inicia el proceso.

        pendientes = [0]

        for i in range(len(df)-1):
            x = slopee(df.index[i],df.Precio[i],df.index[i+1],df.Precio[i+1])
            pendientes.append(x)

        df['pendiente'] = pendientes

        DIFF_SEÑAL_ENTRADA = (datetime.utcnow() - buyprices.index[-1]).total_seconds()

        if DIFF_SEÑAL_ENTRADA < LOCALIDAD_SEÑAL_ENTRADA and df.pendiente[-10:-1].mean() > PENDIENTE_ENTRADA:
            buy_order = client.create_order(
                        symbol=PAIR,
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1
                    )
            tiempo_entrada = datetime.utcnow()
            precio_entrada = float(buy_order['fills'][0]['price'])
            buy_id = buy_order['orderId']

            tiempos_entrada.append(tiempo_entrada)
            precios_entrada.append(precio_entrada)
            buy_ids.append(buy_id)
            print(f'Se realizó una compra al precio: {precio_entrada}, en el momento: {tiempo_entrada}.')

##############  SI EXISTE UNA OPERACIÓN ABIERTA, LA SIGUIENTE CONDICIÓN SE CUMPLE: En ese caso, se debe estar atento a cuándo
##############  cerrar nuestra posición. Ya sea que estemos ganando o perdiendo; se establecen límites de pérdida y de ganancia,
##############  así como también casos en los que las señales de venta se toman en cuenta, tanto en pérdidas como en ganancias.

    if client.get_all_orders(symbol=PAIR, requests_params={'timeout': 30})[-1]['side'] == 'BUY':

        if (df['Precio'][-1] - float(buy_order['fills'][0]['price']))/float(buy_order['fills'][0]['price']) == 0:
            continue 

##############  SEÑALES DE VENTA EN CASO DE TENER PÉRDIDAS.

#############  La primera señal se enfoca en no superar el umbral de pérdidas.       

        elif (df['Precio'][-1] - float(buy_order['fills'][0]['price']))/float(buy_order['fills'][0]['price']) < 0:

            if (df['Precio'][-1] - float(buy_order['fills'][0]['price']))/float(buy_order['fills'][0]['price']) < UMBRAL_PERDIDA_1P:
                sell_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=0.1
                        )
                tiempo_salida = datetime.utcnow()
                precio_salida = float(sell_order['fills'][0]['price'])
                sell_id = sell_order['orderId']
                balance = (precio_salida - precio_entrada)

                tiempos_salida.append(tiempos_salida)
                precios_salida.append(precio_salida)
                sell_ids.append(sell_id)
                balances.append(balance*0.1)
                print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}.  Método 1 en pérdidas.')
                if balance>0:
                    print('La ganancia fue de:', (balance)*0.1, 'dólares')
                    trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance)*0.1,'1P',LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
                                        columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
                    if os.path.exists(path+f'trades_ganados_{pair}.csv'):
                        trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                        mode='a',
                                        index=False,
                                        header=False
                                        )
                    else:
                        trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                        index=False,
                                        header=True,
                                        )
                elif balance==0:
                    print('No hubo pérdidas ni ganancias del trade')
                elif balance<0:
                    print('La pérdida fue de:', (balance)*0.1, 'dólares')
                    trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance)*0.1,'1P',LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
                                        columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
                    if os.path.exists(path+f'trades_perdidos_{pair}.csv'):
                        trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                        mode='a',
                                        index=False,
                                        header=False
                                        )
                    else:
                        trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                        index=False,
                                        header=True,
                                        )
                        
                continue

##############  SEÑALES DE VENTA EN CASO DE TENER GANANCIAS.

        else:

#############  La primera señal se enfoca en no perder todas las ganancias, en caso de existir ganancias. Se permite una pérdida
#############  del 50% de la ganancia máxima.

            if (df['Precio'][-1] - float(buy_order['fills'][0]['price']))/float(buy_order['fills'][0]['price']) > UMBRAL_GANANCIA_1G:

                ganancia_actual = (df['Precio'][-1] - float(buy_order['fills'][0]['price']))

                if ganancia_actual >= ganancia_maxima:
                    ganancia_maxima = ganancia_actual
                else:
                    if (ganancia_actual/ganancia_maxima - 1) < - 0.24: 
                        sell_order = client.create_order(
                                symbol=PAIR,
                                side=Client.SIDE_SELL,
                                type=Client.ORDER_TYPE_MARKET,
                                quantity=0.1
                            )
                        tiempo_salida = datetime.utcnow()
                        precio_salida = float(sell_order['fills'][0]['price'])
                        sell_id = sell_order['orderId']
                        balance = (precio_salida - precio_entrada)

                        tiempos_salida.append(tiempos_salida)
                        precios_salida.append(precio_salida)
                        sell_ids.append(sell_id)
                        balances.append(balance*0.1)
                        print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Método 1 en ganancia.')
                        if balance>0:
                            print('La ganancia fue de:', (balance)*0.1, 'dólares')
                            trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance)*0.1,'1G',LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
                                        columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
                            if os.path.exists(path+f'trades_ganados_{pair}.csv'):
                                trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                                mode='a',
                                                index=False,
                                                header=False
                                                )
                            else:
                                trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                                index=False,
                                                header=True,
                                                )
                        elif balance==0:
                            print('No hubo pérdidas ni ganancias del trade')
                        elif balance<0:
                            print('La pérdida fue de:', (balance)*0.1, 'dólares')
                            trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance)*0.1,'1G',LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
                                        columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
                            if os.path.exists(path+f'trades_perdidos_{pair}.csv'):
                                trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                                mode='a',
                                                index=False,
                                                header=False
                                                )
                            else:
                                trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                                index=False,
                                                header=True,
                                                )
                continue

#############  La segunda señal se enfoca en vender en caso de alcanzar ganancias mínimas del 0.07% y a la vez se da una señal
#############  de venta.        

            if (df['Precio'][-1] - float(buy_order['fills'][0]['price']))/float(buy_order['fills'][0]['price']) <= UMBRAL_MAXIMO_2G and (df['Precio'][-1] - float(buy_order['fills'][0]['price']))/float(buy_order['fills'][0]['price']) > UMBRAL_MINIMO_2G and (datetime.utcnow() - sellprices.index[-1]).total_seconds() < LOCALIDAD_SEÑAL_SALIDA_2G:
                sell_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=0.1
                        )
                tiempo_salida = datetime.utcnow()
                precio_salida = float(sell_order['fills'][0]['price'])
                sell_id = sell_order['orderId']
                balance = (precio_salida - precio_entrada)
                
                tiempos_salida.append(tiempos_salida)
                precios_salida.append(precio_salida)
                sell_ids.append(sell_id)
                balances.append(balance*0.1)
                print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}.  Método 2 en ganancia.')
                if balance>0:
                    print('La ganancia fue de:', (balance)*0.1, 'dólares')
                    trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance)*0.1,'2G',LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
                                        columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
                    if os.path.exists(path+f'trades_ganados_{pair}.csv'):
                        trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                        mode='a',
                                        index=False,
                                        header=False
                                        )
                    else:
                        trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                        index=False,
                                        header=True,
                                        )
                elif balance==0:
                    print('No hubo pérdidas ni ganancias del trade')
                elif balance<0:
                    print('La pérdida fue de:', (balance)*0.1, 'dólares')
                    trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance)*0.1,'2G',LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
                                        columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
                    if os.path.exists(path+f'trades_perdidos_{pair}.csv'):
                        trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                        mode='a',
                                        index=False,
                                        header=False
                                        )
                    else:
                        trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                        index=False,
                                        header=True,
                                    )
                continue
                
#############  La tercer señal se enfoca en vender en caso de alcanzar ganancias máximas del 1.4%

            if (df['Precio'][-1] - float(buy_order['fills'][0]['price']))/float(buy_order['fills'][0]['price']) > UMBRAL_GANANCIA_3G:
                sell_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=0.1
                        )
                tiempo_salida = datetime.utcnow()
                precio_salida = float(sell_order['fills'][0]['price'])
                sell_id = sell_order['orderId']
                balance = (precio_salida - precio_entrada)

                tiempos_salida.append(tiempos_salida)
                precios_salida.append(precio_salida)
                sell_ids.append(sell_id)
                balances.append(balance*0.1)
                print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Método 3 en ganancia.')
                if balance>0:
                    print('La ganancia fue de:', (balance)*0.1, 'dólares')
                    trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance)*0.1,'3G',LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
                                        columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
                    if os.path.exists(path+f'trades_ganados_{pair}.csv'):
                        trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                        mode='a',
                                        index=False,
                                        header=False
                                        )
                    else:
                        trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                        index=False,
                                        header=True,
                                        )
                elif balance==0:
                    print('No hubo pérdidas ni ganancias del trade')
                elif balance<0:
                    print('La pérdida fue de:', (balance)*0.1, 'dólares')
                    trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance)*0.1,'3G',LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
                                        columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
                    if os.path.exists(path+f'trades_perdidos_{pair}.csv'):
                        trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                        mode='a',
                                        index=False,
                                        header=False
                                        )
                    else:
                        trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                        index=False,
                                        header=True,
                                    )
                continue

############# POR ÚLTIMO, VAMOS A GENERAR UNA VENTA SI EL TRADE ESTÁ EN GANANCIAS PERO NO SE HA VENDIDO DESPUÉS DE 10 MINUTOS.

            if ((datetime.utcnow() - tiempo_entrada).total_seconds()/60) > LIMITE_TIEMPO_POSITIVO_4G and (df['Precio'][-1] - float(buy_order['fills'][0]['price']))/float(buy_order['fills'][0]['price']) > UMBRAL_GANANCIA_4G:
                sell_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=0.1
                        )
                tiempo_salida = datetime.utcnow()
                precio_salida = float(sell_order['fills'][0]['price'])
                sell_id = sell_order['orderId']
                balance = (precio_salida - precio_entrada)

                tiempos_salida.append(tiempos_salida)
                precios_salida.append(precio_salida)
                sell_ids.append(sell_id)
                balances.append(balance*0.1)
                print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Método 4 en ganancia.')
                if balance>0:
                    print('La ganancia fue de:', (balance)*0.1, 'dólares')
                    trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance)*0.1,'4G',LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
                                        columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
                    if os.path.exists(path+f'trades_ganados_{pair}.csv'):
                        trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                        mode='a',
                                        index=False,
                                        header=False
                                        )
                    else:
                        trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                        index=False,
                                        header=True,
                                        )
                elif balance==0:
                    print('No hubo pérdidas ni ganancias del trade')
                elif balance<0:
                    print('La pérdida fue de:', (balance)*0.1, 'dólares')
                    trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance)*0.1,'4G',LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
                                        columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
                    if os.path.exists(path+f'trades_perdidos_{pair}.csv'):
                        trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                        mode='a',
                                        index=False,
                                        header=False
                                        )
                    else:
                        trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                        index=False,
                                        header=True,
                                        )

C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - y1) / (x2 - x1).total_seconds()
C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - y1) / (x2 - x1).total_seconds()
C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - y1) / (x2 - x1).total_seconds()


Se realizó una compra al precio: 237.2, en el momento: 2023-06-30 03:38:47.709550.
Se realizó una venta al precio: 238.21, en el momento: 2023-06-30 04:40:40.438117. Método 4 en ganancia.
La ganancia fue de: 0.10100000000000194 dólares


C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - y1) / (x2 - x1).total_seconds()


Se realizó una compra al precio: 238.4, en el momento: 2023-06-30 04:40:44.431343.
Se realizó una venta al precio: 239.5, en el momento: 2023-06-30 04:43:00.149077.  Método 2 en ganancia.
La ganancia fue de: 0.10999999999999943 dólares


C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - y1) / (x2 - x1).total_seconds()


...

C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - y1) / (x2 - x1).total_seconds()
C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - y1) / (x2 - x1).total_seconds()
C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - y1) / (x2 - x1).total_seconds()
C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - y1) / (x2 - x1).total_seconds()
C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - y1) / (x2 - x1).total_seconds()
C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - 

Se realizó una compra al precio: 240.1, en el momento: 2023-06-30 10:16:52.856478.
Se realizó una venta al precio: 241.6, en el momento: 2023-06-30 12:07:01.514623. Método 4 en ganancia.
La ganancia fue de: 0.15000000000000002 dólares


C:\Users\tinma\AppData\Local\Temp\ipykernel_17296\3809730279.py:18: RuntimeWarning: invalid value encountered in double_scalars
  x = (y2 - y1) / (x2 - x1).total_seconds()


Se realizó una compra al precio: 241.8, en el momento: 2023-06-30 12:07:05.496427.
Se realizó una venta al precio: 239.0, en el momento: 2023-06-30 13:36:26.550436.  Método 1 en pérdidas.
La pérdida fue de: -0.28000000000000114 dólares
Se realizó una compra al precio: 239.43, en el momento: 2023-06-30 13:36:34.147226.
Se realizó una venta al precio: 236.8, en el momento: 2023-06-30 13:38:42.024772.  Método 1 en pérdidas.
La pérdida fue de: -0.26299999999999957 dólares
Se realizó una compra al precio: 236.6, en el momento: 2023-06-30 13:39:21.987112.
Se realizó una venta al precio: 233.8, en el momento: 2023-06-30 13:41:59.648438.  Método 1 en pérdidas.
La pérdida fue de: -0.2799999999999983 dólares
Se realizó una compra al precio: 234.2, en el momento: 2023-06-30 13:42:07.429274.
Se realizó una venta al precio: 235.4, en el momento: 2023-06-30 13:43:14.431978.  Método 2 en ganancia.
La ganancia fue de: 0.12000000000000172 dólares
Se realizó una compra al precio: 235.2, en el momento: 2

KeyboardInterrupt: 

In [34]:
sum(balances)

-0.45999999999999364

In [1]:
client.get_account()

NameError: name 'client' is not defined

In [32]:
orders = client.get_all_orders(symbol='BNBUSDT')
orders[-1]['side']

'SELL'

In [25]:
order = client.create_order(
    symbol='BNBUSDT',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=0.1)

In [30]:
order1 = client.create_order(
    symbol='BNBUSDT',
    side=Client.SIDE_SELL,
    type=Client.ORDER_TYPE_MARKET,
    quantity=0.1)

In [24]:
order['fills'][0]['price']

'231.08000000'

In [14]:
order1['fills'][0]['price']

'234.40000000'